In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0331 18:34:13.145998401    1023 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0331 18:34:15.366976154    1023 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [3]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [4]:
# # ran_operators = [int(filename.split('_')[0]) for filename in os.listdir('speedmaps') if filename[0] != '.']
ran_operators = [300]

In [5]:
pbar = tqdm()

0it [00:00, ?it/s]

#### Run all operators

In [6]:
fs_list = fs.ls(f'{utils.GCS_FILE_PATH}rt_trips/')

In [7]:
ran_operators = [int(path.split('rt_trips/')[1].split('_')[0])
                 for path in fs_list
                 if path.split('rt_trips/')[1]]

In [8]:
# for agency in air_joined.calitp_itp_id.unique():
for agency in [4]:
    not_ran_operators = []
    if agency in ran_operators:
        print(f'already ran: {agency}')
        continue
    analysis_date = dt.date(2022, 3, 23) ##wednesday, new tables
    day = str(analysis_date.day).zfill(2)
    month = str(analysis_date.month).zfill(2)
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        delay_to_parquet = rt_day.stop_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet['arrival_time'] = delay_to_parquet.arrival_time.map(lambda x: x.isoformat())
        delay_to_parquet['actual_time'] = delay_to_parquet.actual_time.map(lambda x: x.isoformat())
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        shared_utils.utils.geoparquet_gcs_export(delay_to_parquet,
                                         f'{utils.GCS_FILE_PATH}stop_delay_views/',
                                        f'{agency}_{month}_{day}'
                                        )
        # rt_day.set_filter(start_time='15:00', end_time='19:00')
        # m = rt_day.segment_speed_map(how='low_speeds', size = [1300, 700])
        # shared_utils.utils.geoparquet_gcs_export(rt_day.stop_segment_speed_view,
        #                                          f'{utils.GCS_FILE_PATH}segment_speed_views/',
        #                                         f'{agency}_{month}_{day}_pm_peak'
        #                                         )
        rt_day.rt_trips.to_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{agency}_{month}_{day}_pm_peak.parquet') ## early step towards scaling
        # m.save(f'./speedmaps/{agency}_02_{day}_pm_peak.html')
        # rt_day.stop_segment_speed_view.to_parquet(f'{utils.GCS_FILE_PATH}segment_speed_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
        delay_to_parquet = rt_day.endpoint_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        shared_utils.utils.geoparquet_gcs_export(delay_to_parquet,
                                         f'{utils.GCS_FILE_PATH}endpoint_delay_views/',
                                        f'{agency}_{month}_{day}_pm_peak'
                                        )
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 4...
found parquet
found parquet
found parquet
found parquet


/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:879: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
/opt/conda/lib/python3.9/site-packages/siuba/sql/utils.py:52: SAWarning: Dialect bigquery:bigquery will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metad

could not generate delays for trip 2824040
Columns must be same length as key
could not generate delays for trip 10430020
Columns must be same length as key
could not generate delays for trip 1757020
Columns must be same length as key
could not generate delays for trip 1512020
Columns must be same length as key


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_names': None, 'shape_ids': None, 'direction_id': None, 'direction': None}
shp-33-68_1_ empty!
shp-216-16_0_ empty!
shp-620-01_0_ empty!
shp-21-13_0_ empty!
speed above 80 for shape shp-21-13, dropping
shp-21-20_0_ empty!
shp-52-14_0_ empty!
speed above 80 for shape shp-52-14, dropping
shp-18-62_1_ empty!
shp-99-10_0_ empty!
speed above 80 for shape shp-232-58, dropping
shp-232-58_1_ empty!
shp-7-56_1_ empty!
shp-V-07_0_ empty!
shp-33-14_0_ empty!
shp-70-01_0_ empty!
shp-12-14_0_ empty!
shp-65-10_0_ empty!
shp-28-09_0_ empty!
shp-45-04_0_ empty!
speed above 80 for shape shp-45-04, dropping
shp-6-14_0_ empty!
shp-696-01_0_ empty!
shp-86-01_0_ empty!
shp-684-57_1_ empty!
shp-688-53_1_ empty!
speed above 80 for shape shp-57-62, dropping
shp-57-62_1_ empty!
shp-29-61_1_ empty!
shp-36-09_1_ empty!
shp-62-52_1_ empty!
shp-675-57_1_ empty!
shp-J-07_0_ empty!
shp-88-07_0_ empty!
shp-40-61_1_ empty!
shp-40

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:604: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*ini

complete for agency: 4


### Ongoing issues
* 358 Union City too many trips without shape?

In [ ]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')